서론
- feed forward NN
  - 흐름이 단방향인 신경망
  - 시계열 데이터를 잘 다루지 못한다는 단점
  - 시계열 데이터의 성질을 충분히 학습할 수 없다

# 5.1 확률과 언어 모델

## 5.1.1 word2vec을 확률 관점에서 바라보다

## 5.1.2 언어모델
- 언어모델은 단어 나열에 확률을 부여한다.

<img src="../../../data/deep_learning_2_images/fig 5-4.png" width="500">

- 동시확률은 사후 확률의 총곱
- 사후 확률은 타깃 단어보다 왼쪽에 있는 모든 단어를 맥락으로 했을 때의 확률

# 5.2 RNN이란
- RNN , Recurrent Neural Network

## 5.2.1 순환하는 신경망
- 순환하기 위해서는 닫힌 경로가 필요하다.

<img src="../../../data/deep_learning_2_images/fig 5-6.png" width="200">


## 5.2.2 순환 구조 펼치기

<img src="../../../data/deep_learning_2_images/fig 5-8.png" width="500">

- $h_t = tanh(h_{t-1}W_h+x_tW_x+b)$
- h라는 상태를 가진다.
- $h_t$를 은닉상태(hidden state) 혹은 은닉상태벡터(hidden state vector)라고 한다.

<img src="../../../data/deep_learning_2_images/fig 5-9.png" width="500">


## 5.2.3 BPTT
<img src="../../../data/deep_learning_2_images/fig 5-10.png" width="500">

- Backpropagation Throuhg Time
- 긴 시계열 데이터를 학습할 때의 문제
  - BPTT가 소비하는 컴퓨팅 자원도 증가하기 때문에
  - 역전파 시의 기울기가 불안정해진다. 

## 5.2.4 Truncated BPTT
- 신경망을 적당한 길이로 끊는다.
- 작은 신경망 여러 개로 만든다.
- 역전파의 연결만 끊는다. 순전파의 연결은 그대로

<img src="../../../data/deep_learning_2_images/fig 5-11.png" width="500">

- 반드시 기억할 점은 역전파의 연결은 끊어지지만 순전파의 연결은 끊어지지 않는다는 점
- 데이터를 순서대로 sequential 입력해야 한다.

<img src="../../../data/deep_learning_2_images/fig 5-12.png" width="300">
<img src="../../../data/deep_learning_2_images/fig 5-13.png" width="300">

- 순전파는 이어지고, 역전파는 블록 단위로 진행됨

<img src="../../../data/deep_learning_2_images/fig 5-14.png" width="500">


## 5.2.5 Truncated BPTT의 미니배치 학습
- 데이터를 주는 시작 위치를 각 미니배치의 시작 위치로 옮겨줘야 한다.

<img src="../../../data/deep_learning_2_images/fig 5-15.png" width="500">

- 미니배치 학습을 수행할 때는 각 미니배치의 시작 위치를 오프셋으로 옮겨준 후 순서대로 제공하면 됨.

# 5.3 RNN 구현

<img src="../../../data/deep_learning_2_images/fig 5-17.png" width="550">

- Time RNN 계층 내에서 한 단계의 작업을 수행하는 계층을 'RNN 계층'이라 함.
- T개 단계분의 작업을 한꺼번에 처리하는 계층을 'Time RNN 계층'이라 함.

## 5.3.1 RNN 계층 구현
<img src="../../../data/deep_learning_2_images/e 5-10.png" width="300">

- 미니배치로 처리

<img src="../../../data/deep_learning_2_images/fig 5-18.png" width="300">


forward

<img src="../../../data/deep_learning_2_images/fig 5-19.png" width="500">


In [4]:
import numpy as np

class RNN:
  def __init__(self, Wx, Wh, b):
    self.params = [Wx, Wh, b] # N x H, H x H, N x 1
    self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
    self.cache = None
    
  def forward(self, x, h_prev):
    Wx, Wh, b = self.params
    t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
    h_next = np.tanh(t)
    
    self.cache = (x, h_prev, h_next)
    
    return h_next

backward

<img src="../../../data/deep_learning_2_images/fig 5-20.png" width="500">


In [5]:
def backward(self, dh_next):
  Wx, Wh, b = self.params
  x, h_prev, h_next = self.cache
  
  dt = dh_next * (1 - h_next ** 2)
  db = np.sum(dt, axis=0)
  dWh = np.matmul(h_prev.T, dt)
  dh_prev = np.matmul(dt, Wh.T)
  dWx = np.matmul(x.T, dt)
  dx = np.matmul(dt, Wx.T)
  
  self.grads[0][...] = dWx
  self.grads[1][...] = dWh
  self.grads[2][...] = db
  
  return dx, dh_prev
  

## 5.3.2 Time RNN 계층 구현

<img src="../../../data/deep_learning_2_images/fig 5-21.png" width="500">
<img src="../../../data/deep_learning_2_images/fig 5-22.png" width="500">



In [6]:
class TimeRNN:
  def __init__(self, Wx, Wh, b, stateful=False):
    self.params = [Wx, Wh, b]
    self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
    self.layers = None
    
    self.h, self.dh = None, None
    self.stateful = stateful # 은닉상태를 인계 받을지 여부
    
  def forward(self, xs):
    Wx, Wh, b = self.params
    N, T, D = xs.shape
    D, H = Wx.shape
    
    self.layers = []
    hs = np.empty((N, T, H), dtype='f')
    
    if not self.stateful or self.h is None:
      self.h = np.zeros((N, H), dtype='f') # 영행렬로 초기화
      
    for t in range(T):
      layer = RNN(*self.params)
      self.h = layer.forward(xs[:, t, :], self.h)
      hs[:, t, :] = self.h
      self.layers.append(layer)
      
    return hs

Time RNN 계층의 역전파

<img src="../../../data/deep_learning_2_images/fig 5-23.png" width="500">
<img src="../../../data/deep_learning_2_images/fig 5-24.png" width="500">

- 순전파에서는 출력이 2개로 분기됨. 
- 따라서 역전파에서는 각 기울기가 합산되어 전해짐

In [9]:
def backward(self, dhs):
  Wx, Wh, b = self.params
  N, T, H = dhs.shape
  D, H = Wx.shape
  
  dxs = np.empty((N, T, D), dtype='f')
  dh = 0
  grads = [0, 0, 0]
  for t in reversed(range(T)):
    layer = self.layers[t]
    dx, dh = layer.backward(dhs[:, t, :] + dh) # 합산된 기울기 ( 위로 올라간거 + 앞에로 간거랑)
    dxs[:, t, :] = dx
    
    for i, grad in enumerate(layer.grads):
      grads[i] += grad
      
  for i, grad in enumerate(grads):
    self.grads[i][...] = grad
  self.dh = dh
  
  return dxs

# 5.4 시계열 데이터 처리 계층 구현
- RNN을 이용해서 '언어 모델'을 구현하는 것
- 시계열 데이터를 처리하는 계층을 몇 개 더 추가
- RNNLM

## 5.4.1 RNNLM의 전체그림
<img src="../../../data/deep_learning_2_images/fig 5-25.png" width="500">

- Embedding : 단어 ID를 단어의 분산표현으로 변환

<img src="../../../data/deep_learning_2_images/fig 5-26.png" width="500">

- RNN 계층은 "you say"라는 맥락을 '기억'하고 있다는 사실.
- RNN은 "you say"라는 과거의 정보를 응집된 은닉 상태 벡터로 저장해두고 있다.
- RNNLM은 지금까지 입력된 단어를 기억하고 그것을 바탕으로 다음에 출현할 단어를 예측
- RNN 계층이 과거에서 현재로 데이터를 계속 흘려보내줌으로써 과거의 정보를 인코딩해 저장(기억)할 수 있다.

## 5.4.2 Time 계층 구현
- Time RNN, Time Embedding, Time Affine

<img src="../../../data/deep_learning_2_images/fig 5-27.png" width="500">

- Time Affine
  - 단순히 Affine 계층 T개를 이용하는 방식 대신 행렬 계산으로 한꺼번에 처리하는, 효율 좋은 방식으로 구현
  
<img src="../../../data/deep_learning_2_images/fig 5-28.png" width="500">


- Time Softmax with Loss 
  - $x_0 , x_1$ 등의 데이터는 아래층에서부터 전해지는 '점수'를 나타냄
  - $t_0, t_1$ 등의 데이터는 정답 레이블

<img src="../../../data/deep_learning_2_images/fig 5-29.png" width="500">
<img src="../../../data/deep_learning_2_images/e 5-11.png" width="300">



# 5.5 RNNLM 학습과 평가

## 5.5.1 RNNLM 구현

<img src="../../../data/deep_learning_2_images/fig 5-30.png" width="500">

In [3]:
import sys
sys.path.append('../../modules/Part2/')
import numpy as np
from common.time_layers import *

class SimpleRnnlm:
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn
    
    # 가중치 초기화
    embed_W = (rn(V, D) / 100).astype('f')
    rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')
    rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
    rnn_b = np.zeros(H).astye('f')
    affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
    affine_b = np.zeros(H).astye('f')
    
    # 계층생성
    self.layers = [
      TimeEmbedding(embed_W),
      TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
      TimeAffine(affine_W, affine_b)
    ]
    self.loss_layer = TimeSoftmaxWithLoss()
    self.rnn_layer = self.layers[1]
    
    # 모든 가중치와 기울기를 리스트에 모은다.
    self.params, self.grads = [], []
    for layer in self.layers:
      self.params += layer.params
      self.grads += layer.grads
      
  def forward(self, xs, ts):
    for layer in self.layers:
      xs = layer.forward(xs)
      
    loss = self.loss_layer.forward(xs, ts)
    return loss
  
  def backward(self, dout=1):
    dout = self.loss_layer.backward(dout)
    for layer in reversed(self.layers):
      dout = layer.backward(dout)
      
    return dout
  
  def reset_state(self):
    self.rnn_layer.reset_state()

## 5.5.2 언어 모델의 평가
- 언어모델은 과거 단어로부터 다음에 출현할 단어의 확률분포를 출력
- 퍼플렉서티 perplxity , 혼란도
  - 확률의 역수
  - 작을수록 좋음
  - 분기 수 , number of branches
  - 정보이론 분야에서는 퍼플렉서티를 기하평균 분기 수라고도 한다.

<img src="../../../data/deep_learning_2_images/e 5-12.png" width="200">
<img src="../../../data/deep_learning_2_images/e 5-13.png" width="200">

## 5.5.3 RNNLM의 학습코드
- PTB 데이터셋
- 이번에 구현한 RNNLM은 PTB 데이터셋 전부를 대상으로 학습하면 전혀 좋은 결과를 낼 수 없다.
- 처음 1000개 단어만 이용

In [ ]:
import sys
sys.path.append('../../modules/Part2/')
import matplotlib.pyplot as plt
import numpy as np
from common.optimizer import SGD
from dataset import ptb
from simple_rnnlm import SimpleRnnlm

# hyperparameter
batch_size = 10
word2vec_size = 100
hidden_size = 100
time_size = 5
lr = 0.1
max_epoch = 100

# read train data ([:1000])
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:1000]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1]
ts = corpus[1:]
data_size = len(xs)
print(f"말뭉치 크기: {corpus_size}, 어휘 수: {vocab_size}")

max_iters = data_size // (batch_size * time_size) # 1000 // (10 * 5) -> 20
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

# create model
model = SimpleRnnlm(vocab_size, word2vec_size, hidden_size)
optimizer = SGD(lr)

# 각 미니배치에서 샘플을 읽기 시작, 시작위치를 계산
jump = (corpus_size - 1) // batch_size # 1000 // 10 -> 100
offsets = [i * jump for i in range(batch_size)] # [0, 100, 200, ..., 900, 1000]

for epoch in range(max_epoch): # [0, ..., 100]
  for iter in range(max_iters): # [0, ..., 20]
    # 미니배치 획득
    batch_x = np.empty((batch_size, time_size), dtype='i')
    batch_t = np.empty((batch_size, time_size), dtype='i')
    for t in range(time_size):
      for i, offset in enumerate(offsets):
        batch_x[i, t] = xs[(offset + time_idx) % data_size]
        batch_t[i, t] = ts[(offset + time_idx) % data_size]
      time_idx += 1
      
      
    
